In [11]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib inline

In [12]:
imgpoints = [] # 2D points in image plane
objpoints = [] # 3D points in real world space

objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2) # x, y coordinate

corner_images = []
def draw_corners(chessimage):
    
    nx = 9
    ny = 6
    
    img = cv2.imread(chessimage)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        corner_images.append(img)
    else:
        corner_images.append(img)

In [13]:
import os

for file in os.listdir("camera_cal/"):
    draw_corners("camera_cal/"+file)

In [17]:
corner_frame = plt.figure(figsize=(18,9))

#for i in range(0, len(corner_images)):
    #axis = corner_frame.add_subplot(4,5,i+1)
    #plt.xticks(np.array([]))
    #plt.yticks(np.array([]))
    #axis.imshow(corner_images[i])
#corner_frame.tight_layout(h_pad=0, w_pad=0)
#corner_frame.savefig('corners.png')

In [44]:
undistorted_images = []

def undistort_img(path, imgname, objpoints, imgpoints):
    img = cv2.imread(path+"/"+imgname)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[0:2], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #cv2.imwrite(path+"/"+imgname[0:imgname.find('.')]+"u.jpg", undist)
    #undistorted_images.append(undist)
    return undist

for file in os.listdir("camera_cal/"):
    undist = undistort_img("camera_cal",file, objpoints, imgpoints)
    cv2.imwrite("camera_cal/"+file[0:file.find('.')]+"u.jpg", undist)
    undistorted_images.append(undist)

    
#undistorted_frame = plt.figure(figsize=(18,9))

#for i in range(0, len(undistorted_images)):
#    axis = undistorted_frame.add_subplot(4,5,i+1)
#    plt.xticks(np.array([]))
#    plt.yticks(np.array([]))
#    axis.imshow(undistorted_images[i])
#undistorted_frame.tight_layout(h_pad=0, w_pad=0)
#undistorted_frame.savefig('undistorted_images.png')

In [53]:
path = "test_images"
file = "test6.jpg"
img = mpimg.imread(path+"/"+file)

#plt.imshow(img)

#plt.plot(730, 453,".")
#plt.plot(1102, 668,".")
#plt.plot(304, 668,".")
#plt.plot(622, 453,".")

def warp(img):
    img_size = (img.shape[1], img.shape[0])

    #top right, bottom right, bottom left, top left
    
    
    src = np.float32([[768,478],[1115,684],[285, 684],[575, 478]])
    #src = np.float32([[730,453],[1102,668],[304, 668],[622, 453]])
    
    #dst = np.float32([[1115,235],[1115,684],[285, 684],[285, 235]])
    dst = np.float32([[1115,135],[1115,718],[285, 718],[285, 135]])
    
    M = cv2.getPerspectiveTransform(src, dst)

    Minv = cv2.getPerspectiveTransform(dst, src)

    warped_img = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped_img
    
    
undist = undistort_img(path, file, objpoints, imgpoints)
warped_img = warp(undist)
cv2.imwrite(path+"/warped_img.jpg", warped_img)

True